In [1]:
import numpy as np
import math
import itertools
from functools import cache

In [2]:
n = 10
q = 2
x=np.random.randint(0,q,(n),dtype=np.ubyte)

In [3]:
def get_deletion_ball(x, t, N=-1):
    if N == -1:
        return np.array(list(set(itertools.combinations(x, x.size-t))))
    else:
        result = set()
        for c in itertools.combinations(x, x.size-t):
            result.add(c)
            if len(result) == N:
                break

        return np.array(list(result))



In [4]:
r=1
D_x_1 = get_deletion_ball(x,r)
print(f'For {x} size {n} vector of alphabet {q} we get a radius {r} deletion ball of size {D_x_1.shape[0]}')

For [0 0 1 1 0 1 0 0 0 0] size 10 vector of alphabet 2 we get a radius 1 deletion ball of size 5


In [5]:
def get_composition_vector(x):
    k = np.zeros(q,dtype=int)
    for i in x:
        k[i] += 1
    return k

def get_ordering(x):
    k = get_composition_vector(x)
    t = np.argsort(-k, kind='stable') # -k so it will be in descending order
    c = k[t]
    return t, c

def number_of_runs(x):
    runs = 1
    prev_x = x[0]
    for i in x[1:]:
        if i != prev_x:
            runs += 1
        prev_x = i

    return runs

@cache
def get_maximal_deletion_ball_size(n, t, q=2):
    if n < t or t < 0:
        return 0
    if q == 1:
        return 1

    size = 0
    for i in range(t+1):
        size += math.comb(n-t, i) * get_maximal_deletion_ball_size(t, t-i, q-1)
    return size

@cache
def get_maximal_insertion_ball_size(n, t, q=2):
    if n < 0 or t < 0:
        return 0
    size = 0
    for i in range(t+1):
        size += math.comb(n+t, i)*int(math.pow((q-1),i))
    return size

@cache
def get_maximal_number_of_common_subsequences(n, t, q=2):
    if n <= t or t <= 0:
        return 0
    return get_maximal_deletion_ball_size(n,t,q) - get_maximal_deletion_ball_size(n-1,t,q) + get_maximal_deletion_ball_size(n-2,t-1,q)

In [6]:
k_x = get_composition_vector(x)
t_x, l_x = get_ordering(x)
print(k_x)
print(t_x)
print(l_x)
print(x, number_of_runs(x))
print(get_maximal_deletion_ball_size(n,1))
print(get_maximal_insertion_ball_size(n,1))
print(get_maximal_number_of_common_subsequences(n,1))
print(get_maximal_number_of_common_subsequences(n,2))
print(get_maximal_number_of_common_subsequences(7,3, 3))

[7 3]
[0 1]
[7 3]
[0 0 1 1 0 1 0 0 0 0] 5
10
12
2
16
24


In [7]:
def get_threshold(n, t, order_comp, q=2):
    for i in range(q):
        N = get_maximal_number_of_common_subsequences(n-i-1,t-i,q)
        if N < order_comp[i]:
            return i

def get_u_a_i(subsequences, a, i=1):
    return subsequences[:,subsequences[0,:] == a][1:,:]


def reconstruct_x_from_subsequences(n, subsequences, q=2, print_steps=False):
    t = n-subsequences.shape[0]
    reconstruction = np.array([], dtype=int)
    while t >= 1:
        order_perm, order_comp = get_ordering(subsequences[0])
        j = get_threshold(n, t, order_comp, q)
        reconstruction = np.concatenate((reconstruction, order_perm[:j+1]))
        if print_steps:
            print(j, reconstruction)

        n = n-j-1
        t = t-j    
        N = get_maximal_number_of_common_subsequences(n,t,q)+1
        subsequences = get_u_a_i(subsequences, order_perm[j])
        subsequences = subsequences[:,:N]
        
    return np.concatenate((reconstruction, subsequences.T[0]))
            
    

In [8]:
n = 100
t = 5
q = 20


x=np.random.randint(0,q,(n),dtype=np.ubyte)
N = get_maximal_number_of_common_subsequences(n,t,q)+1
print(N)
D_x = get_deletion_ball(x, t, N)

while D_x.shape[0] < N:
    print('1')
    x=np.random.randint(0,q,(n),dtype=np.ubyte)
    D_x = get_deletion_ball(x, t, N)

7376657


In [9]:
subsequences = D_x.T
print(subsequences.shape)
# print(get_next_subsequnces(subsequences,0).shape)
# print(get_next_subsequnces(subsequences,0))
reconstructed_x = reconstruct_x_from_subsequences(n, subsequences, q)
print(f'Reconstructed {reconstructed_x}')
print(f'From {x}')
print(np.array_equal(x, reconstructed_x))

(95, 7376657)
Reconstructed [ 2  8 14  7  3  9  2 18  8 13  7 14  5  7  9  1  7 11 19 16 11 19  7 15
  9  0  2 19 11  3 19  1  6 13 11  7  2  9  8 19  8  2 16 11  1 13 10  9
 17  3  2  5 12 12 17 17  2  9 14 11 18 14  5  7  7 17 15 18 13  3 15  5
  7  4  2 18 11 10 14  1 11  4  0  1 13 11  2  2 12 19  8  4 15  4 15  2
  8  4  2  7]
From [ 2  8 14  7  3  9  2 18  8 13  7 14  5  7  9  1  7 11 19 16 11 19  7 15
  9  0  2 19 11  3 19  1  6 13 11  7  2  9  8 19  8  2 16 11  1 13 10  9
 17  3  2  5 12 12 17 17  2  9 14 11 18 14  5  7  7 17 15 18 13  3 15  5
  7  4  2 18 11 10 14  1 11  4  0  1 13 11  2  2 12 19  8  4 15  4 15  2
  8  4  2  7]
True
